# Create an intake catalogue using ACCESS-MOPPeR

In this tutorial we are showing how to create an intake-esm catalogue to access ACCESS raw model output. To do so we are using the ACCESS-MOPPeR command line tool. MOPPeR was developed to post-process raw model output to CMIP style variables. However, as part of the workflow is the mapping of raw output to well-described variables it can be used to create a curated list of the raw output to use as base for an intake catalogue. 

In [1]:
from dask.distributed import Client

client = Client("tcp://127.0.0.1:41671")
client

<Client: 'tcp://127.0.0.1:41671' processes=4 threads=12, memory=46.00 GiB>

In [2]:
import intake
import xarray as xr
import pandas as pd
import numpy as np

In [ ]:
mopenv = '/g/data/ua8/Working/packages/ACCESS-MOPPeR/mopper_env/bin'

In [7]:
# MOPPeR will soon be available in the hh5 conda environment, in the meantime we installed it in a custom conda environment
! source /g/data/ua8/Working/packages/ACCESS-MOPPeR/mopper_env/bin/activate

To run this tutorial we are using some raw model output from a AUS2200 run. NB the UM data output has been converted to netcdf but the files are still stored in the same directory structure as they were output by the model.

In [10]:
! /g/data/ua8/Working/packages/ACCESS-MOPPeR/mopper_env/bin/mopdb intake -f /g/data/v45/cc6171/experiments/ECL-2016_smallshift/netcdf/u-cs142  -v AUS2200 -m 20160603 -a testrun

Opened database /g/data/ua8/Working/packages/ACCESS-MOPPeR/mopper_env/lib/python3.10/site-packages/mopdata/access.db successfully
Variable list for umnsa_cldrad_ successfully written
Variable list for umnsa_mdl_ successfully written
Found more than 1 definition for fld_s08i225:
[('tsl', 'CM2', 'CMIP6_Lmon', 'mon'), ('tsl', 'ESM1.5', 'CMIP6_Lmon', 'mon')]
 Using tsl from CMIP6_Lmon
Variable list for umnsa_slv_ successfully written
Variable list for umnsa_spec_ successfully written
Derived variables: {'va24', 'prw', 'theta24', 'refl24', 'wa24', 'sifllatstop', 'ta24', 'ua24', 'hus24'}
Changing cl-AUS2200_A1hr units from 1 to %
Changing rainmxrat-AUS2200_A1hr units from kg kg-1 to 1
Changing grplmxrat-AUS2200_A1hr units from kg kg-1 to 1
Variable: rinum- has no associated cmor_table
Variable: rho- has no associated cmor_table
Changing lmask-AUS2200_fx units from 1 to %
Changing siconca-AUS2200_A1hr units from 1 to %
Changing evspsbl_Pt-AUS2200_A1hr realm from atmos to land
Changing evspsbl

This command will create 3 files:
* the main intake catalogue intake_test.yaml, 
* the intake-esm catalogue for the selected source intake_test.json,
* and finally a list of all the files path and attributes catalogue.csv.xz

We can now open the catalogue.

In [3]:
cat = intake.open_catalog('/g/data/ua8/Working/packages/ACCESS-MOPPeR/intake_demo.yaml')

Usually intake would assume that each file contains one variable, which is not the case for raw model output, this is while we also defined a "yaml" catalogue file veen if we are only delaing with one datasets. In the catalogue definition we let intake knows that some of the fields have multiple values:
* variable: the roginal variable name
* map_var: the cmip style variable name where available
* standard_name: the variable standard_name where available

NB only `variable` will have a complete list of all the variables available, but as the name in the raw model output can be obscured we added the other two lists to faciliate queries.

As an alternative the intake-esm json catalogue can be accessed directly with a few adjustments.

In [4]:
print(list(cat))

['demo']


In [5]:
demo = cat['demo']
#testrun = intake.open_esm_datastore(
#    "/g/data/ua8/Working/packages/ACCESS-MOPPeR/intake_testrun.json",
#    columns_with_iterables=["variable", "map_var", "standard_name"],
#)
demo.esmcat.has_multiple_variable_assets

True

In [6]:
demo.unique()['mapvar']

['NAV',
 'theta',
 'huslev1',
 'phalf',
 'pslev1',
 'zfull',
 'snw',
 'zmla',
 'z0',
 'siconca',
 'rss',
 'rsut',
 'rsdt',
 'rsdsdir',
 'rsdsdiff',
 'rsds',
 'rls',
 'rlut',
 'rlds',
 'uas',
 'vas',
 'hfss_Pt',
 'hfss',
 'evspsbl_Pt',
 'evspsbl',
 'estot',
 'eow',
 'hfls_Pt',
 'hfls',
 'tasmin',
 'tas',
 'tasmax',
 'huss',
 'tdps',
 'tauu',
 'tauv_Pt',
 'tauv',
 'wsgmax10m',
 'mrso',
 'cw',
 'mrsol',
 'tsl',
 'clmxro',
 'psl',
 'ts',
 'ps',
 'evspsblpot',
 'wsgmax10m_max',
 'reflmax',
 'pralsprof',
 'pralsns',
 'cllow',
 'clmed',
 'clhigh',
 'flashrate',
 'storm',
 'nflash',
 'amdry',
 'amwet',
 'lwp',
 'clivi']

In [7]:
subset = demo.search(mapvar=['uas'], frequency='1hr')
subset.df

,path,experiment,realm,frequency,variable,mapvar,standard_name,date,units,calculation,cell_methods
0,/g/data/v45/cc6171/experiments/ECL-2016_smalls...,demo2,atmos,1hr,[fld_s03i209],uas,eastward_wind,201606030000,m s-1,NaN,area: time: mean
1,/g/data/v45/cc6171/experiments/ECL-2016_smalls...,demo2,atmos,1hr,[fld_s03i209],uas,eastward_wind,201606030101,m s-1,NaN,area: time: mean
2,/g/data/v45/cc6171/experiments/ECL-2016_smalls...,demo2,atmos,1hr,[fld_s03i209],uas,eastward_wind,201606030201,m s-1,NaN,area: time: mean
3,/g/data/v45/cc6171/experiments/ECL-2016_smalls...,demo2,atmos,1hr,[fld_s03i209],uas,eastward_wind,201606030301,m s-1,NaN,area: time: mean
4,/g/data/v45/cc6171/experiments/ECL-2016_smalls...,demo2,atmos,1hr,[fld_s03i209],uas,eastward_wind,201606030401,m s-1,NaN,area: time: mean
...,...,...,...,...,...,...,...,...,...,...,...
115,/g/data/v45/cc6171/experiments/ECL-2016_smalls...,demo2,atmos,1hr,[fld_s03i209],uas,eastward_wind,201606071901,m s-1,NaN,area: time: mean
116,/g/data/v45/cc6171/experiments/ECL-2016_smalls...,demo2,atmos,1hr,[fld_s03i209],uas,eastward_wind,201606072001,m s-1,NaN,area: time: mean
117,/g/data/v45/cc6171/experiments/ECL-2016_smalls...,demo2,atmos,1hr,[fld_s03i209],uas,eastward_wind,201606072101,m s-1,NaN,area: time: mean
118,/g/data/v45/cc6171/experiments/ECL-2016_smalls...,demo2,atmos,1hr,[fld_s03i209],uas,eastward_wind,201606072201,m s-1,NaN,area: time: mean


This includes the entire timeseries we want to get a smaller subset only the data for the 5th of June after 8pm. We can do this in two ways:
* using a regex to select on date=r"201606052.*"
* or by selecting all the files in between two dates.

In this specific case using the first method it's esier but we will demo both.

In [8]:
subset = subset.search(date=r"201606052.*")
subset.df

,path,experiment,realm,frequency,variable,mapvar,standard_name,date,units,calculation,cell_methods
0,/g/data/v45/cc6171/experiments/ECL-2016_smalls...,demo2,atmos,1hr,[fld_s03i209],uas,eastward_wind,201606052001,m s-1,NaN,area: time: mean
1,/g/data/v45/cc6171/experiments/ECL-2016_smalls...,demo2,atmos,1hr,[fld_s03i209],uas,eastward_wind,201606052101,m s-1,NaN,area: time: mean
2,/g/data/v45/cc6171/experiments/ECL-2016_smalls...,demo2,atmos,1hr,[fld_s03i209],uas,eastward_wind,201606052201,m s-1,NaN,area: time: mean
3,/g/data/v45/cc6171/experiments/ECL-2016_smalls...,demo2,atmos,1hr,[fld_s03i209],uas,eastward_wind,201606052301,m s-1,NaN,area: time: mean


In [9]:
dsets = subset.to_dataset_dict(parallel=True)


--> The keys in the returned dictionary of datasets are constructed as follows:
	'realm.frequency.mapvar'


In [10]:
[k for k in dsets.keys()]

['atmos.1hr.uas']

In [11]:
dsets['atmos.1hr.uas']

<xarray.Dataset> Size: 7GB
Dimensions:             (time: 4, lat: 2120, lon: 2600, time_0: 4, lon_0: 2600,
                         lat_0: 2121, depth: 4, time_1: 4, bnds: 2)
Coordinates: (12/75)
  * time                (time) datetime64[ns] 32B 2016-06-05T21:00:00 ... 201...
  * lat                 (lat) float64 17kB -48.79 -48.77 ... -6.852 -6.832
  * lon                 (lon) float64 21kB 114.3 114.3 114.3 ... 165.7 165.7
  * time_0              (time_0) datetime64[ns] 32B 2016-06-05T20:30:00 ... 2...
  * lon_0               (lon_0) float64 21kB 114.3 114.3 114.3 ... 165.7 165.7
  * lat_0               (lat_0) float64 17kB -48.8 -48.78 ... -6.842 -6.822
    ...                  ...
    fld_s09i212         (time, lat, lon) float32 88MB dask.array<chunksize=(4, 1060, 1300), meta=np.ndarray>
    fld_s09i213         (time, lat, lon) float32 88MB dask.array<chunksize=(4, 1060, 1300), meta=np.ndarray>
    fld_s09i214         (time, lat, lon) float32 88MB dask.array<chunksize=(4, 1060, 1300), meta=np.ndarray>
    fld_s09i215         (time, lat, lon) float32 88MB dask.array<chunksize=(4, 1060, 1300), meta=np.ndarray>
    fld_s09i217         (time_0, lat, lon) float32 88MB dask.array<chunksize=(4, 1060, 1300), meta=np.ndarray>
    fld_s16i222         (time_0, lat, lon) float32 88MB dask.array<chunksize=(4, 1060, 1300), meta=np.ndarray>
Dimensions without coordinates: bnds
Data variables:
    fld_s03i209         (time_1, time, time_0, lat, lon_0) float32 1GB dask.array<chunksize=(1, 4, 4, 1060, 1300), meta=np.ndarray>
Attributes: (12/14)
    Conventions:                     CF-1.6
    source:                          Data from Met Office Unified Model
    um_version:                      12.2
    intake_esm_vars:                 ['fld_s03i209']
    intake_esm_attrs:experiment:     demo2
    intake_esm_attrs:realm:          atmos
    ...                              ...
    intake_esm_attrs:mapvar:         uas
    intake_esm_attrs:standard_name:  eastward_wind
    intake_esm_attrs:units:          m s-1
    intake_esm_attrs:cell_methods:   area: time: mean
    intake_esm_attrs:_data_format_:  netcdf
    intake_esm_dataset_key:          atmos.1hr.uas

In [ ]:
subset = testrun.search(variable=['fld_s03i209','fld_s03i210'], frequency='1hr', date=r"201606052.*")

Please note that what we got so far is not a very good result, intake-esm will try to sleect only the variables we passed however these don't exists in the dataset as we passed the mapped variables not the original names. We still selected the correct files but we need to tell intake-esm the right names. We can check the names in the mapping that was saved as the same time as the intake catalogues.

In [ ]:
from functools import partial

In [ ]:
def _preprocess(ds,varlist):
    return ds[varlist]

In [ ]:
selvar = partial(_preprocess, varlist=['fld_s03i209', 'fld_s03i210'])

In [ ]:
dsets = subset.to_dataset_dict(preprocess=selvar)

In [ ]:
dsets['atmos.1hr']

In [ ]:
dsets['ocean.mon.uo']

## Calculating derived variables

In [ ]:
We will now show ...

In [ ]:
from intake_esm import DerivedVariableRegistry

Initiate the `derived variable registry`

In [ ]:
dvr = DerivedVariableRegistry()

Then we define a function to calculate wind_speed and use the register to decorate it

In [ ]:
@dvr.register(variable='wind_speed', query={'map_var': ['uas', 'vas']})
def calc_wind_speed(ds, uasname, vasname):
    ds['wind_speed'] = np.sqrt(ds[uaname] ** 2 + ds[vaname] ** 2)
    ds['wind_speed'].attrs = {'units': "m s-1",
                              'long_name': "Surface wind speed'",
                              'standard_name': "wind_speed",
                              'comment': "Derived using intake-esm from testrun experiment"}
    return ds